<div style="text-align: center;">
    <h1> <font style="bold"> Analisis Predictivo Avanzado </font></h1>
    <h2><font style="bold">Trabajo práctico </font></h2>
    <h3><font style="bold">Integrantes:</font></h3>
</div>
<div style="display: flex; justify-content: center;">
    <h4><ul>
        <li>Noguera Abril</li>
        <li>Arbues Lucas</li>
        <li>Alfie Agustin</li>
        </ul>
    </h4>
</div>

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Funciones: 
from funciones import encoding, balance, Optimization

In [19]:
df = pd.read_pickle('df.pkl')

## Encoding:

In [22]:
dfOHE = encoding(df[['category_id', 'domain_id', 'item_id','name','conversion','ROW_ID']], ['category_id', 'domain_id', 'item_id','name'], 'TE').merge(
    encoding(df.drop(['category_id', 'domain_id', 'item_id','name','conversion'], axis=1), ['listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp'], 'OHE'), 
    left_index=True, right_index=True, how='inner')
dfOHE['ROW_ID'] = dfOHE['ROW_ID_x']
dfOHE =dfOHE.drop(['ROW_ID_x','ROW_ID_y'], axis = 1)

In [ ]:
dfLE = encoding(df, ['category_id', 'domain_id', 'item_id','name', 'listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp'], 'LE')

In [ ]:
dfOE = encoding(df, ['category_id', 'domain_id', 'item_id','name', 'listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp'], 'OE')

In [8]:
dfFE = encoding(df, ['category_id', 'domain_id', 'item_id','name', 'listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp'], 'FE')

In [9]:
dfTE = encoding(df, ['category_id', 'domain_id', 'item_id','name', 'listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp'], 'TE')

# Exploración e Implementación del Modelo 

Una vez finalizado el analisis exploratorio de datos (EDA), se procede a la exploracion e implementacion del modelo.

- Train: 80%
- Val: 20%
- Test: de testeo.

In [24]:
df = dfOHE

In [26]:
# Split into training and evaluation samples
train_data = df[df["ROW_ID"].isna()]
test_data = df[df["ROW_ID"].notna()]

In [27]:
train_data = train_data.drop(columns=["ROW_ID"])
X_train, X_val, y_train, y_val = train_test_split(train_data.drop(columns=["conversion"]), train_data['conversion'], test_size=0.2)

In [28]:
X_test = test_data.drop(columns=["conversion", "ROW_ID"])

## Balanceo:

In [29]:
print((y_train == 1).value_counts())

conversion
False    131196
True      13412
Name: count, dtype: int64


In [30]:
# OS, US, LT, NM y SM
X_train, y_train = balance(X_train, y_train, 'NM')

## Optimización

In [31]:
model = RandomForestClassifier(n_jobs=-1, verbose=1, oob_score=True)

In [32]:
grid = {
            'max_depth': stats.randint(1,20),
              'n_estimators': stats.randint(10, 200),
              'min_samples_split': stats.randint(2, 20),
             }

In [33]:
search_spaces = {
    'n_estimators': (10, 200),
    'max_depth': (1, 32),
    'min_samples_split': (2, 20),
}

### Random Search:

In [34]:
optimizer = Optimization(model=model, param_grid=grid, n_iter=10, cv=5, random_state=42)

In [35]:
%%capture
RS = optimizer.random_search(X_train, y_train)

In [ ]:
best_params_random = RS.best_params_
print("Best hyperparameters (Random Search):", best_params_random)

Best hyperparameters (Random Search): {'max_depth': 7, 'min_samples_split': 16, 'n_estimators': 156}


In [115]:
RS.best_score_

0.7404699738903394

In [68]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19211 entries, 180761 to 199971
Data columns (total 81 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   category_id                         18733 non-null  float64
 1   domain_id                           18971 non-null  float64
 2   item_id                             0 non-null      float64
 3   name                                19211 non-null  float64
 4   available_quantity                  19211 non-null  int64  
 5   avg_gmv_item_domain_30days          19211 non-null  float64
 6   avg_gmv_item_sel                    19211 non-null  float64
 7   avg_gmv_seller_bday                 19211 non-null  float64
 8   avg_qty_orders_item_domain_30days   19211 non-null  float64
 9   avg_qty_orders_item_sel_30days      19211 non-null  float64
 10  avg_si_item_sel_30day               19211 non-null  float64
 11  free_shipping                       1921

In [65]:
# Predict on the evaluation set
y_pred = RS.predict_proba(X_test)[:,1]
y_pred

ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# Make the submission file
submission_df = pd.DataFrame({"ROW_ID": test_data["ROW_ID"], "conversion": y_pred})
submission_df["ROW_ID"] = submission_df["ROW_ID"].astype(int)
submission_df.to_csv("xg_boostRS.csv", sep=",", index=False)